In [42]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np

In [46]:
# lendo como string o conteúdo de um arquivo sql
sql_file_path = "G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\DADOS\\415 - Repositório de Dados\\Tratamento Github\\Ecossistema-dados\\Tratamento de tabelas do BQ\\perfil_remuneracao\\Auxiliar\\RAIS_remuneracao_publicos_privados.sql" 
with open(sql_file_path, "r") as file:
    sql_string = file.read()


In [47]:
## Acessando pelo pacote da BD

df=bd.read_sql(sql_string, billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 976/976 [00:00<00:00, 2633.16rows/s]


In [48]:
df

,ano,flag_publico_privado,variavel,categoria,media_remuneracao
0,2015,privado,tipo_vinculo,CLT,2047.508295
1,2004,privado,tipo_vinculo,CLT,875.875558
2,2001,privado,tipo_vinculo,CLT,699.824617
3,2005,privado,tipo_vinculo,CLT,934.536467
4,2011,privado,tipo_vinculo,CLT,1467.130364
...,...,...,...,...,...
971,2004,publico,grau_instrucao,Até Ensino Superior Completo,2359.961409
972,2012,publico,grau_instrucao,Até Ensino Superior Completo,4422.203366
973,2013,publico,grau_instrucao,Até Ensino Superior Completo,4748.715776
974,2015,publico,grau_instrucao,Até Ensino Superior Completo,5483.889229


In [50]:
## O privado vem com a série inteira, vou consertar essa parte ao consertar a série

df = df[df['ano']>2002]


In [53]:
df['ano'].unique()

<IntegerArray>
[2015, 2004, 2005, 2011, 2020, 2013, 2014, 2006, 2009, 2010, 2019, 2003, 2018,
 2007, 2021, 2008, 2016, 2017, 2012]
Length: 19, dtype: Int64

In [54]:
### Criando função para deflacionar a variável de remuneração
### Vou me basear no IPCA acumulado de dezembro de 2022 para realização do cálculo

## adicionar coluna de fator de correção pra cada ano
## depois faz a conta

os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\DADOS\\415 - Repositório de Dados\\Documentos auxiliares")

In [55]:
ipca = pd.read_csv("ipca.csv",sep=";",decimal=",")

In [21]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ano                     28 non-null     int64  
 1   IPCA acumulado dez/ano  28 non-null     float64
 2   It                      28 non-null     float64
 3   Fator de correcao       28 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 1.0 KB


In [66]:
df1= df.merge(ipca[['Ano','Fator de correcao']], right_on='Ano',left_on='ano')

In [67]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,Ano,Fator de correcao
0,2015,privado,tipo_vinculo,CLT,2047.508295,2015,1.441141
1,2015,privado,tipo_vinculo,Avulso,1956.729128,2015,1.441141
2,2015,privado,nivel_governo,Outros,2014.787251,2015,1.441141
3,2015,privado,poderes,Outros,2014.787251,2015,1.441141
4,2015,privado,tipo_vinculo,Diretor,9715.239893,2015,1.441141
...,...,...,...,...,...,...,...
819,2012,publico,grau_instrucao,Até Mestrado,5816.083506,2012,1.797445
820,2012,publico,grau_instrucao,Até Doutorado,10377.552735,2012,1.797445
821,2012,publico,grau_instrucao,Até Fundamental,1477.950509,2012,1.797445
822,2012,publico,grau_instrucao,Até Ensino Médio,2067.034228,2012,1.797445


In [68]:
df1['media_remuneracao_ajustada']= df1['media_remuneracao']* df1['Fator de correcao']

In [59]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,Ano,Fator de correcao,media_remuneracao_ajustada
0,2015,privado,tipo_vinculo,CLT,2047.508295,2015,1.441141,2950.747211
1,2015,privado,tipo_vinculo,Avulso,1956.729128,2015,1.441141,2819.921675
2,2015,privado,nivel_governo,Outros,2014.787251,2015,1.441141,2903.591589
3,2015,privado,poderes,Outros,2014.787251,2015,1.441141,2903.591589
4,2015,privado,tipo_vinculo,Diretor,9715.239893,2015,1.441141,14001.026075
...,...,...,...,...,...,...,...,...
819,2012,publico,grau_instrucao,Até Mestrado,5816.083506,2012,1.797445,10454.091333
820,2012,publico,grau_instrucao,Até Doutorado,10377.552735,2012,1.797445,18653.082268
821,2012,publico,grau_instrucao,Até Fundamental,1477.950509,2012,1.797445,2656.535037
822,2012,publico,grau_instrucao,Até Ensino Médio,2067.034228,2012,1.797445,3715.380735


In [69]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')



In [70]:
df1.columns

Index(['ano', 'flag_publico_privado', 'variavel', 'categoria',
       'media_remuneracao', 'Ano', 'Fator de correcao',
       'media_remuneracao_ajustada'],
      dtype='object')

In [72]:
df1 = df1[['ano','flag_publico_privado', 'variavel', 'categoria', 'media_remuneracao',
       'fator_correcao', 'media_remuneracao_ajustada']]

In [71]:
df1=df1.rename(columns={'Fator de correcao':'fator_correcao'})

In [73]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824 entries, 0 to 823
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ano                         824 non-null    Int64  
 1   flag_publico_privado        824 non-null    object 
 2   variavel                    824 non-null    object 
 3   categoria                   824 non-null    object 
 4   media_remuneracao           824 non-null    float64
 5   fator_correcao              824 non-null    float64
 6   media_remuneracao_ajustada  824 non-null    float64
dtypes: Int64(1), float64(3), object(3)
memory usage: 52.3+ KB


In [74]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
bigquery.SchemaField('flag_publico_privado','STRING',description='Se se refere ao setor publico ou privado'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]


In [76]:
table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos_privados')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=3aa41642-08ac-4581-bedd-7fc2effe3bfe>

In [75]:
#para excluir tabelas
client.delete_table('perfil_remuneracao.RAIS_remuneracao_publicos_privados', not_found_ok=True)